<center>![alt text](images/shutterstock_712597270-2.jpg)</center>

In [ ]:
import pandas as pd
import requests
import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,7)

In [ ]:
url = 'https://api.coinmarketcap.com/v1/ticker/?limit=0'
r = requests.get(url)
df = pd.read_json(r.text)
df.head()

In [ ]:
df[df['symbol']=="MIOTA"]

In [ ]:
df.iloc[:10]

### Top ten ranked cryptos

In [ ]:
top_10 = df[df['rank']<=10]
top_10.set_index('name', inplace=True)
#top_10.loc['Stellar']

In [ ]:
top_10.plot.scatter(x='market_cap_usd', y='24h_volume_usd', loglog=True, figsize=(10,10));

### Join price df with my holding spreadsheet

In [ ]:
have = pd.read_csv('example_crypto_holdings.csv')

In [ ]:
has = have.join(df.set_index('name'), on='navn', how='left').set_index('id')
has['worth'] = has.amount*has.price_usd

# Filter out coins with no trade volume
has = has[has['24h_volume_usd'].notnull()]
has

### Find USD/NOK exchange rate

In [ ]:
url_exchange = 'https://api.fixer.io/latest?symbols=USD,NOK'
r = requests.get(url_exchange)
NOK_USD = r.json()['rates']['NOK']
has['worth_nok'] = has['worth']*NOK_USD
has[['price_usd', 'worth','worth_nok']], has['worth_nok'].sum()

In [ ]:
has.dtypes

### Visualize my portfolio

In [ ]:
has['worth_nok'].plot(kind='bar');

In [ ]:
has['24h_volume_usd'].plot(kind='bar') #logy=True

In [ ]:
has[['percent_change_1h','percent_change_24h','percent_change_7d']].plot(kind='bar');

In [ ]:
df1 = has
df1 = has[['percent_change_1h','percent_change_24h','percent_change_7d']].apply(lambda r: r*df1.amount*df1.price_usd)*NOK_USD/100
df1.plot(kind='bar'), df1.sum();

In [ ]:
df2 = has
df2 = has[['percent_change_7d', 'percent_change_24h']].apply(lambda r:  r*(df2.amount*df2.price_usd))/100

df2 = df2.apply(lambda r: has.worth-r, axis=0)
df2['worth'] = has.worth
df2 = df2*NOK_USD
df2.plot(kind='bar');

In [ ]:
df2.plot.pie('worth', figsize=(12,12));